# FRAP calculation

This Script is trying to calculate FRAP data from given .csv files. It then export it as a summary csv containing all FRAPs of one experiment. For this to work filestructure needs to be like this

Experiment ->

    Condition -> 
    
        FRAP-imamge1-bleach.csv
        FRAP-image1-reference.csv
        FRAP-image1-referencexor.csv
        FRAP-image1-background
        .
        .
        .
        

In [23]:
#import section
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

## File handling

In [24]:
#filepath to this folder
origin = os.getcwd()
#print(origin)

#get the experiment number
Exp_no= os.path.basename(os.path.abspath(os.path.join(origin, os.pardir)))

In [25]:
#Function gets the subdirectory names and dreturns them
def get_subdirnames (input):
    
    folders=[]
    
    for dirpath, dirnames, filenames in os.walk(input):
        for folder in dirnames:
            if folder ==".ipynb_checkpoints":
                continue
            folders.append(os.path.join(dirpath,folder))
            
    return folders

#gets a list of all the .csv files in afolder
def get_files(input):
    filelist=[]
    fileextension=".csv"
    
    for dirpath, dirnames, filenames in os.walk(input):
        for files in filenames:
            if files.endswith(fileextension):
                filelist.append(os.path.join(input,files))
    
    return filelist

In [26]:
def split_basename(name):
    
    #get basename
    basename=os.path.basename(name)
    
    #remove fileending
    basename=basename[:-4]
    #print(basename)
    #creats list [FRAP no, measurmenttype (Backgroun, reference etc.)]
    
    element=basename.split("-")
    
    return element

def sortfiles(filelist):
   
    file_ends=["background.csv","reference.csv","xor.csv","bleach.csv"]
    suffixdic={}
    pathdic={}
    firstrun=True
    
    for end in file_ends:
        #print(end)
        for path in filelist:
            
            if path.endswith(end):
                name=split_basename(path)
                #print(name)
                suffixdic[name[0]]={name[1]:path}
                #display(suffixdic)
        
        if firstrun==True:
            pathdic=suffixdic
            firstrun=False
       
        else:
            pathdic={k:{**pathdic[k], **suffixdic[k]} for k in suffixdic}
        suffixdic={}
    #display(pathdic)
    
    return(pathdic)

In [27]:
sortedPaths={}
subdirlist=get_subdirnames(origin)
#print("subdirpaths:\n",subdirlist)

for item in subdirlist:
    condition=os.path.basename(item)
    files=get_files(item)
    sortedPaths[condition]=sortfiles(files)
    
print(sortedPaths.keys())

dict_keys(['E200824_FRAP-2BP-BSA', 'E200824_FRAP-2BP-DMEM', 'E200824_FRAP-2BP-OA', 'E200824_FRAP-BSA', 'E200824_FRAP-DMEM', 'E200824_FRAP-DMSO-BSA', 'E200824_FRAP-DMSO-OA', 'E200824_FRAP-OA', 'E200824_FRAP-Palmo-BSA', 'E200824_FRAP-Palmo-DMEM', 'E200824_FRAP-Palmo-OA'])


## Calculating Normalisations and FRAP curves

In [28]:
#Calculate Phair-normalisation from Ref, Refxor and bleach
def calc_Phair(df):
    
    #Create a Backgroundsubstracted data of Reference, Reference of proximity of the bleach (xor)
    #Put them in a new collumn (Always expand no data is overwritten)
    df['Ref-Back']=df.reference-df.background #column no. 6
    df['Refxor-Back']=df.referencexor-df.background #column no. 7
    df['Bleach-Back']= df.bleach - df.background #column no. 8
    
    #Calculate the mean intensity of the five prebleach images(0:5) of the columns 6,7,8 
    premean=df.iloc[0:5,5:9].mean()# ReferencePrebleach mean = premean[0], BleachPrebleach mean = premean[1]
    #print(premean)
   
    MeanPreBleach=premean[2]
    MeanPreReference=premean[0]
    MeanPreReferencexor=premean[1]
    
    #Calculates Phair normalization ratio of pre-bleach mean Reference divided by each reference
    #=> How much signal ist lost by just photobleaching
    #Multiply that ratio with the Bleach values of all time points
    # do that for both references
    df['Phair'] = (MeanPreReference/df['Ref-Back'])*(df['Bleach-Back']/MeanPreBleach) #no9
    df['Phairxor']= (MeanPreReferencexor/df['Refxor-Back'])*(df['Bleach-Back']/MeanPreBleach) #no10
    
    return (df)

In [29]:
def calc_0_1(df):
    
    # Make the minimum zero and the maximum 1 by substracting the min and divide by max
    df['Featurescaled']= (df['Phair']-df['Phair'].min())/(df['Phair'].max()-df['Phair'].min()) #no11
    df['Featurescaledxor']= (df['Phairxor']-df['Phairxor'].min())/(df['Phairxor'].max()-df['Phairxor'].min()) #no12
    
    return (df)

## Plotting the FRAP-Curves

This is only for checking if all the curves were detected and created correctly. It is saved in the original files folder as pdf collecting all the figures of one FRAP-experiment.

In [30]:
def FRAP_plot(title, df, save):
    
    # Save is the folder in which the original files are to be found
    
    imsavepath= (save+"_FRAP-curves.pdf")
    
    #define y and x axis
    
    x= df.index
    y1= df.bleach
    y2= df.reference
    y3= df.referencexor
    y4= df.background
    y5= df.Phair
    y6= df.Featurescaled
    y7= df.Phairxor
    y8= df.Featurescaledxor
    
    with PdfPages(imsavepath) as export_pdf:
        
        #First image References bleach and Background
        plt.scatter(x,y1, label ='Bleach')
        plt.scatter(x,y2, label ='Reference')
        plt.scatter(x,y3, label ='Reference-XOR')
        plt.scatter(x,y4, label ='Background')
    
        plt.xlabel('time [sec]')
        plt.ylabel('Intensity')
    
        plt.title(title+'FRAP-Curve')
    
        plt.legend()
        #plt.show()
        export_pdf.savefig()
        plt.close()
        
        #Second image Phair Featurscaled with normal reference
        plt.scatter(x,y5, label ='Phair')
        plt.scatter(x,y6, label ='Featurescaled')    
        plt.xlabel('time[sec]')
        plt.ylabel('Relative intensity')
        plt.title(title+'Normalized-Curve')
    
        plt.legend()
        #plt.show()
        export_pdf.savefig()
        plt.close()
        
        #Third image Phair Featurescaled with xor derived reference
        plt.scatter(x,y7, label ='Phair-XOR')
        plt.scatter(x,y8, label ='Featurescaled-XOR')    
        plt.xlabel('time[sec]')
        plt.ylabel('Relative intensity')
        plt.title(title+'Normalized-Curve')
    
        plt.legend()
        #plt.show()
        export_pdf.savefig()
        plt.close()
        
    

## Creation of pandas dataframe

In [31]:
def load_FRAP(dic, Fname):
    
    path_dic=dic[Fname]
    fields={'X','Y'}
    first_run=True
        
    for key in path_dic:
        #print(key)
        csv_path=path_dic[key]    
           # print (csv_path)
        
        with open(csv_path) as file:
            tempDF=pd.read_csv(file,skipinitialspace=True, usecols=fields, index_col=False)
            
        if first_run==True:
            DF=tempDF
            DF.columns=['time',key]
            DF.set_index('time')
            first_run=False
                
        else:
            DF[key]=tempDF['Y']
            DF.set_index('time')
    DF=calc_Phair(DF)
    DF=calc_0_1(DF)
    #display(DF)
    
    spath= os.path.join(os.path.abspath(os.path.join(csv_path, os.pardir)), Fname)
    #print(spath)
    DF.to_csv(spath+"_analysis.csv")
    FRAP_plot(Fname,DF, spath)
    
    return DF

In [32]:
def create_summary(PandaList, name):
    
    
    SummaryDF={}
    Summaryxor={}
    
    for key in PandaList:
        
        df=PandaList[key]
        xor=PandaList[key]
        SummaryDF[key]=df.iloc[5:,10].values
        Summaryxor[key]=xor.iloc[5:,11].values
  
    time= df.time.values
    time= time[5:]
    startT=min(time)
    time[:]= [t - startT for t in time]
   
    path=os.path.join(name+"_summary.csv")
    xorpath=os.path.join(name+"_xor_summary.csv")
    
    #display(SummaryDF)
    FinalDF=pd.DataFrame.from_dict(SummaryDF)
    FinalDFxor=pd.DataFrame.from_dict(Summaryxor)
    
    #display(FinalDF)
    
    FinalDF.index=time
    FinalDFxor.index=time
    
    FinalDF.to_csv(path)
    FinalDFxor.to_csv(xorpath)

In [33]:

for condition in sortedPaths:
    print(condition)
    DFDic={}
    for key in sortedPaths[condition]:
        print(key)
        DFDic[key]=(load_FRAP(sortedPaths[condition],key))
    create_summary(DFDic, condition)
    
print("DONE") 

E200824_FRAP-2BP-BSA
FRAP01Ch01
FRAP02Ch01
FRAP03Ch01
FRAP04Ch01
FRAP05Ch01
FRAP06Ch01
E200824_FRAP-2BP-DMEM
FRAP01Ch01
FRAP02Ch01
FRAP03Ch01
FRAP04Ch01
FRAP05Ch01
FRAP06Ch01
E200824_FRAP-2BP-OA
FRAP01Ch01
FRAP02Ch01
FRAP03Ch01
FRAP04Ch01
FRAP05Ch01
FRAP06Ch01
E200824_FRAP-BSA
FRAP01Ch01
FRAP02Ch01
FRAP03Ch01
FRAP04Ch01
FRAP05Ch01
FRAP06Ch01
E200824_FRAP-DMEM
FRAP01Ch01
FRAP02Ch01
FRAP03Ch01
FRAP04Ch01
FRAP05Ch01
E200824_FRAP-DMSO-BSA
FRAP01Ch01
FRAP02Ch01
FRAP03Ch01
FRAP04Ch01
FRAP05Ch01
FRAP06Ch01
E200824_FRAP-DMSO-OA
FRAP01Ch01
FRAP02Ch01
FRAP03Ch01
FRAP04Ch01
FRAP05Ch01
FRAP06Ch01
E200824_FRAP-OA
FRAP01Ch01
FRAP02Ch01
FRAP03Ch01
FRAP04Ch01
FRAP05Ch01
FRAP06Ch01
E200824_FRAP-Palmo-BSA
FRAP01Ch01
FRAP02Ch01
FRAP03Ch01
E200824_FRAP-Palmo-DMEM
FRAP01Ch01
FRAP02Ch01
FRAP03Ch01
E200824_FRAP-Palmo-OA
FRAP01Ch01
FRAP02Ch01
FRAP03Ch01
FRAP04Ch01
FRAP05Ch01
DONE
